# Séminaire IMT Grand-Est

# Introduction à l'apprentissage automatique - TP6 exercice 1

### SVM sur données synthétiques

<br> 

Prenez connaissance de la [documentation scikit-learn sur les SVM](http://scikit-learn.org/stable/modules/svm.html).

On utilisera la [classe SVC](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).

Notez la valeur par défaut de l'hyperparamètre $C$, et les fonctions noyau disponibles (ainsi que leurs paramètres).

<br>

Dans les questions suivantes, vous séparerez les bases de données
entre bases d'apprentissage (80% de la base initiale) et base de test (20%) en utilisant `model_selection.train_test_split`, et vous calculerez un score de
classification sur la base de test.

<br>

On commence par charger les bibliothèques utiles et définir une fonction de visualisation:

In [ ]:
from sklearn import datasets, model_selection, preprocessing, model_selection, svm
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.colors import Normalize

%matplotlib inline

def plot_classif_result_SVM(X,y,clf,title):
    cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA'])
    cmap_bold = ListedColormap(['#FF0000', '#00FF00'])    
    cmap2 = ListedColormap(['#FF8888', '#FFAAAA', '#AAFFAA', '#88FF88'])  
    
    h=0.01 # step size in the mesh
    
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Zdf = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Zdfbin = (np.abs(Zdf)<=1)   # 0 if inside margin,  1 if outside 
    Color=np.zeros(Z.shape)  # colors for each region 
    for i in range(len(Z)):
        if (Z[i]):
            if Zdfbin[i]: Color[i]=2
            else: Color[i]=3
        else:
            if Zdfbin[i]: Color[i]=1 
            else: Color[i]=0
                
    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    Color = Color.reshape(xx.shape)    
    plt.figure(figsize=[8,8])
    plt.pcolormesh(xx, yy, Color, cmap=cmap2)
    
    # Plot also the training points
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold,
                edgecolor='k', s=20)
    
    # Plot the support vectors:
    plt.scatter(X[clf.support_, 0], X[clf.support_, 1], c=y[clf.support_], cmap=cmap_bold,edgecolor='k',s=80, marker='*')    
    
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.title(title);
    plt.axis("equal")

Commencez par un jeu de données de 1000 points dans deux classes linéairement séparables, obtenu avec `make_blobs` (comme fait dans un des TP précédents). 

Entraînez une SVM à noyau linéaire, et visualisez les vecteurs supports à l'aide de la fonction `plot_classif_result_SVM`.

Affichez également le score de classification sur la base test, ainsi que le nombre de vecteurs supports pour chaque classe.

Même question avec deux classes non séparables (augmentez la valeur de `cluster_std` dans `make_blobs`).

In [ ]:

# génération dataset
# faire plusieurs exécutions: les deux classes ne sont pas toujours linéairement séparables
X_dataset, y_dataset = datasets.make_blobs(n_features=2, centers=2, cluster_std=1.0, n_samples=1000)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X_dataset,y_dataset,test_size=.2)

# affichage dataset train+test
plt.figure()
plt.scatter(X_dataset[:, 0], X_dataset[:, 1], marker='o', c=y_dataset,
            s=25, edgecolor='k')
plt.title('dataset')
plt.axis("equal")

# votre code ici: entraînant SVM et visualisation


 Reprenez le jeu de données synthétique `make_moons`:
 ```python
X_dataset, y_dataset = datasets.make_moons(noise=0.2, n_samples=1000)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_dataset,y_dataset,test_size=.2)
```
 Visualisez
  le résultat de la SVM linéaire, et vérifiez que diminuer la valeur de l'hyperparamètre $C$ augmente le nombre de vecteurs supports (et change la surface de séparation). Vous essaierez les valeurs de $C$ suivantes: 0.01, 1, 100.


In [ ]:

# génération dataset
X_dataset, y_dataset = datasets.make_moons(noise=0.2, n_samples=1000)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X_dataset,y_dataset,test_size=.2)

# affichage dataset train+test
plt.figure()
plt.scatter(X_dataset[:, 0], X_dataset[:, 1], marker='o', c=y_dataset,
            s=25, edgecolor='k')
plt.title('dataset')

# Votre code ici:


Observez le résultat de la classification avec un noyau
  gaussien (RBF), et l'évolution selon différentes valeurs du
  paramètre `gamma` (on gardera $C=1$).

In [ ]:
# votre code ici:


Pour le noyau RBF et la valeur par défaut de $\gamma$, discutez l'influence de $C$.

In [ ]:
# votre code ici:


 Pour trouver une valeur optimale aux hyperparamètres $\gamma$ et $C$ par validation croisée, on dispose de la fonction [GridSearchCV](scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) rencontrée au TP précédent.

Cette fonction va calculer (par défaut dans la version actuelle de `sklearn`) le score de 3-fold cross validation pour différentes valeurs des paramètres.
Visualisez les résultats de la cellule suivante:

In [ ]:
gamma_range=10**(np.arange(-3.,4.5,.5))
C_range=10**(np.arange(-3.,3.5,.5)) 
parameters = { 'gamma': gamma_range, 'C': C_range }
SVM = svm.SVC(kernel='rbf')
gridsearch=model_selection.GridSearchCV(SVM, parameters)
gridsearch.fit(X_train,y_train)
print("Meilleur estimateur trouvé:")
print(gridsearch.best_estimator_)
print("Meilleurs paramètres:")
print(gridsearch.best_params_)

scores = gridsearch.cv_results_['mean_test_score'].reshape(len(C_range),len(gamma_range))
plt.figure(figsize=[10,10])
plt.imshow(scores)
plt.xlabel('gamma')
plt.ylabel('C')
plt.xticks(np.arange(len(gamma_range)), gamma_range, rotation=45)
plt.yticks(np.arange(len(C_range)), C_range)
plt.colorbar()
plt.show();


Quel est le score de classification sur la base de test du meilleur classifieur identifié?

In [ ]:
# votre code ici:


Comparez au résultat de la classification de la base de test par les algorithmes de classification aux 1,5, 10 plus proches voisins, à la régression logistique, au classifieur naïf gaussien et au perceptron multicouches.

_Indication_. Vous utiliserez les paramètres pour le perceptron multicouches:

``` hidden_layer_sizes=(7,),max_iter=1000,solver='lbfgs',alpha=0.001,activation='tanh' ```

In [ ]:
from sklearn import  neighbors, naive_bayes, linear_model, neural_network

# votre code ici:
